In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

import altair as alt

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [2]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_shs.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Month_no']= df['datetime'].dt.month
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Month_no'].astype(str)
df.head(3)

,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Month_no,Year_no,Period
<Ticker>,,,,,,,,,,,,,,,,,
SHS,20210506,28.5,28.9,27.7,27.8,5119800,28.5,28.9,27.7,27.8,0,27300,11000,2021-05-06,5,2021,2021-5
SHS,20210505,27.6,29.0,27.5,28.5,6722263,27.6,29.0,27.5,28.5,160000,27300,11000,2021-05-05,5,2021,2021-5
SHS,20210504,27.0,27.6,26.1,27.6,6313234,27.0,27.6,26.1,27.6,0,0,36200,2021-05-04,5,2021,2021-5


In [3]:
df =df[['<Close>','datetime','<Volume>','Period','Year_no']]
df = df.head(1500)

In [4]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name='SHS'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [17]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'STOCK Violin chart', height= 800, width= 1200)

In [6]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
SHS,6.4,2015-05-15,1045200,2015-5,2015
SHS,6.7,2015-05-14,377900,2015-5,2015
SHS,6.5,2015-05-13,376978,2015-5,2015


In [7]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
SHS,28.5,2021-05-05,6722263,2021-5,2021,2.517986
SHS,27.6,2021-05-04,6313234,2021-5,2021,-3.157895
SHS,27.4,2021-04-29,4550766,2021-4,2021,-0.724638


In [8]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [9]:
df['Daily_return'].describe()

count    1499.000000
mean       -0.053699
std         2.942366
min        -9.090909
25%        -1.467895
50%         0.000000
75%         1.408451
max        14.150943
Name: Daily_return, dtype: float64

In [10]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [11]:
df['<Close>'].describe()

count    1499.000000
mean       11.845697
std         6.000900
min         4.400000
25%         7.200000
50%        10.200000
75%        14.600000
max        32.400000
Name: <Close>, dtype: float64

In [12]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [13]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                      mode ='lines', name='Span_7_Rolling_vol'))

fig.add_trace(go.Scatter(x=df.datetime, y=df['<Close>'], 
                        mode='lines+markers', name='Price'))

In [14]:
df['Volumn'].head(30).describe()

count    30.000000
mean      9.392068
std       4.262177
min       4.020723
25%       6.415491
50%       8.590924
75%      11.378201
max      23.709894
Name: Volumn, dtype: float64

In [15]:
df['Volumn'].head(10).describe()

count    10.000000
mean      6.791986
std       3.136423
min       4.020723
25%       4.948942
50%       6.005474
75%       6.881837
max      14.892845
Name: Volumn, dtype: float64

In [16]:
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return,Volumn
<Ticker>,,,,,,,
SHS,28.5,2021-05-05,6722263,2021-5,2021,2.517986,6.722263
SHS,27.6,2021-05-04,6313234,2021-5,2021,-3.157895,6.313234
SHS,27.4,2021-04-29,4550766,2021-4,2021,-0.724638,4.550766
